In [1]:
# Import Dependencies
import pandas as pd
import dask.dataframe as dd
import numpy as np
import requests
import time
import os
from pprint import pprint
from pathlib import Path

output_path = os.path.join("Exports")
os.makedirs('Exports', exist_ok=True)
base_url = "https://microdata.worldbank.org/index.php/api/tables/data/fcv/wld_2021_rtfp_v02_m"


In [2]:
# Fetch data using requests
all_data = []
params = {
    'format': 'json',
    'limit': 1000,
    'offset': 0
}

while True:
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data and 'data' in data:
            all_data.extend(data['data'])
            if len(data['data']) < params['limit']:
                break
            else:
                params['offset'] += params['limit']
        else:
            break
    else:
        print("Error:", response.status_code)
        break

print(f"Total records fetched: {len(all_data)}")


Total records fetched: 505428


In [3]:
# Convert JSON to pandas, then to dask
data_df= dd.from_pandas(pd.json_normalize(all_data), npartitions=1)
print(data_df.head())


  ISO3      country   adm1_name adm2_name    mkt_name    lat    lon  \
0  AFG  Afghanistan  Badakhshan      Argo  Badakhshan  37.04  70.46   
1  AFG  Afghanistan  Badakhshan      Argo  Badakhshan  37.04  70.46   
2  AFG  Afghanistan  Badakhshan      Argo  Badakhshan  37.04  70.46   
3  AFG  Afghanistan  Badakhshan      Argo  Badakhshan  37.04  70.46   
4  AFG  Afghanistan  Badakhshan      Argo  Badakhshan  37.04  70.46   

                   geo_id       DATES  year  ...  l_yogurt c_yogurt  \
0  gid_370400000704600000  2007-01-01  2007  ...                      
1  gid_370400000704600000  2007-02-01  2007  ...                      
2  gid_370400000704600000  2007-03-01  2007  ...                      
3  gid_370400000704600000  2007-04-01  2007  ...                      
4  gid_370400000704600000  2007-05-01  2007  ...                      

  inflation_yogurt trust_yogurt o_food_price_index  h_food_price_index  \
0                                             0.61                0.62  

In [4]:
# Drop columns with unwanted prefixes
prefixes = ['h_', 'l_']
columns_to_drop = [col for col in data_df.columns if col.startswith(tuple(prefixes))]
data_df = data_df.drop(columns=columns_to_drop)

# Drop additional unnecessary columns
additional_columns = [
    'start_dense_data', 'last_survey_point', 'adm1_name', 'adm2_name',
    'lat', 'lon', 'geo_id', 'year', 'month', 'components'
]
existing_columns = [col for col in additional_columns if col in data_df.columns]
if existing_columns:
    data_df = data_df.drop(columns=existing_columns)


In [5]:
# Save to disk in parquet format
data_df.to_parquet('Exports/cleaned_data.parquet', write_index=False)
parquet_path = os.path.join(output_path, 'cleaned_data.parquet')
data_df.to_parquet(parquet_path, write_index=False)


In [6]:
# Compute and preview the full result (optional, may use large memory)
# result = data_df.compute()
data_df.head()


,ISO3,country,mkt_name,DATES,currency,data_coverage,data_coverage_recent,index_confidence_score,spatially_interpolated,apples,...,inflation_yam,trust_yam,o_yogurt,c_yogurt,inflation_yogurt,trust_yogurt,o_food_price_index,c_food_price_index,inflation_food_price_index,trust_food_price_index
0,AFG,Afghanistan,Badakhshan,2007-01-01,AFN,34.58,75.42,0.92,0,,...,,,,,,,0.61,0.61,,7.5
1,AFG,Afghanistan,Badakhshan,2007-02-01,AFN,34.58,75.42,0.92,0,,...,,,,,,,0.61,0.61,,7.5
2,AFG,Afghanistan,Badakhshan,2007-03-01,AFN,34.58,75.42,0.92,0,,...,,,,,,,0.61,0.61,,7.5
3,AFG,Afghanistan,Badakhshan,2007-04-01,AFN,34.58,75.42,0.92,0,,...,,,,,,,0.61,0.65,,7.5
4,AFG,Afghanistan,Badakhshan,2007-05-01,AFN,34.58,75.42,0.92,0,,...,,,,,,,0.66,0.7,,7.5


In [7]:
# Save to disk in CSV format
#data_df.to_csv('Exports/cleaned_data.csv', single_file=False, index=False)
csv_path = os.path.join(output_path, 'cleaned_data.csv')
data_df.to_csv(csv_path, single_file=False, index=False)

['c:\\Users\\Valky\\Desktop\\Class\\Project-3--World-Food-Bank\\Exports\\cleaned_data.csv\\0.part']